# KNIT6 OWL Data Analysis

## Imports
This series of notebooks all need a common set of imports which are defined in [Common Imports](./KNIT6_common_imports.ipynb)

**slice_name** is defined in this step. If you would like to change the slice_name, edit [Common Imports](./KNIT6_common_imports.ipynb)

In [ ]:
%run "./KNIT6_common_imports.ipynb"

# First things first (Install necessary packages)

### For extracting data from `*.pcap` files

In [ ]:
!conda install -y -q -c conda-forge scapy

### For graphing

In [ ]:
!conda install -y  -c conda-forge -c plotly jupyter-dash

### Boring but necessary imports

In [ ]:
from pathlib import Path
from mflib import owl_data

import pandas as pd

# Critical for showing data inline 
import plotly.io as pio
pio.renderers.default = 'iframe'

# Input data information 

- slice name 
- location of local `*.pcap` files (give root directory path from which a recursive search will run)
- csv file (for extracted data)

In [ ]:
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%Y%m%d%H%M")

#slice_name="MFLibKNIT6"
root_data_dir = f'/home/fabric/work/owl_output/{slice_name}'
csv_path = f'/home/fabric/work/owl_output/{slice_name}_{current_time}.csv'

## If csv file does not yet exist and `*.pcap` files need to be parsed, run the following

### List all the .pcap files under the root data dir (recursive search)

In [ ]:
pcap_files = owl_data.list_pcap_files(root_data_dir)
print(pcap_files)

### Extract entries from PCAP and add to the csv file

Args:

- `pcap_files`: list of pcap file paths
- `outfile (default="out.csv")`: path to output csv file
- `append_csv (default=False)`: if set to True, append to the existing csv file of the same name; if False, it will warn and exit in case there is already a csv file of the same name.
- `verbose (default=False)`: if True, it will print out the content of pcap files during the extraction process

In [ ]:
owl_data.convert_pcap_to_csv(pcap_files, outfile=csv_path)

# Create an analyzer instance

It creates a pandas DataFrame from the csv file. Incomplete rows are dropped. 

The only argument is the path to csv file. CSV file should have 5 columns:
- source IP
- sent timestamp (epoch time)
- destination IP
- destinatin timestamp (epoch time)
- sequence number
- latency


In [ ]:
owl_data_analyzer = owl_data.OwlDataAnalyzer(csv_path)

## Get slice data

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)  
except Exception as e:
    print(f"Exception: {e}")
    
nodes = slice.get_nodes()
# for node in nodes:
#     print(node.get_name())
    

### Print nodes information

In [ ]:
sites_df = owl_data_analyzer.find_node_locations(nodes)
sites_df

### But it should be prettier ... print a map
Try hovering over each point. 

In [ ]:
owl_data_analyzer.print_map(sites_df)

# Analyze the latency data

## (optional) Check the content of pandas.Dataframe

In [ ]:
df = owl_data_analyzer.get_dataframe()
df

## Filter by source/destination

In [ ]:
node_names = ["Node1", "Node2", "Node3"]

node1 = slice.get_node(name="Node1")
node2 = slice.get_node(name="Node2")
node3 = slice.get_node(name="Node3")
node1_ip = owl_data_analyzer.list_experiment_ip_addrs(node1)[0]
node2_ip = owl_data_analyzer.list_experiment_ip_addrs(node2)[0]
node3_ip = owl_data_analyzer.list_experiment_ip_addrs(node3)[0]

print(node1_ip, node2_ip, node3_ip)

## Get statistics

In [ ]:
owl_data_analyzer.summarize_data(src_node=node1, dst_node=node2)

In [ ]:
owl_data_analyzer.summarize_data(src_node=node2, dst_node=node1)

In [ ]:
owl_data_analyzer.summarize_data(src_node=node3, dst_node=node2)

In [ ]:
owl_data_analyzer.summarize_data(src_node=node1, dst_node=node3)

## Graph the latency over time

In [ ]:
owl_data_analyzer.graph_latency_data(node1, node2)

In [ ]:
owl_data_analyzer.graph_latency_data(node2, node1)

In [ ]:
owl_data_analyzer.graph_latency_data(node3, node2)

In [ ]:
owl_data_analyzer.graph_latency_data(node1, node3)